In [26]:
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [27]:
import nltk
nltk.download('punkt')

# This downloads the rest of TextBlob's required corpora
!python -m textblob.download_corpora


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [28]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from urllib.parse import urlparse
from textblob import TextBlob
from collections import Counter

In [30]:
import praw
from urllib.parse import urlparse
import sys

# --------- CONFIGURE THIS PART --------------
CLIENT_ID = 'Rb_a*************j_pw'
CLIENT_SECRET = 'TB***************CIag'
USER_AGENT = "windows:RedditProfileScraper:v1.0 (by /u/DevelopmentNatural62)"
# --------------------------------------------

In [31]:

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)


In [32]:

def extract_username(url):
    """Extract the Reddit username from the profile URL."""
    path = urlparse(url).path.strip('/')
    return path.split('/')[1] if path.startswith('user/') else None

def fetch_user_data(username, limit=30):
    """Fetch recent posts and comments made by the Reddit user."""
    redditor = reddit.redditor(username)
    posts, comments = [], []

    for submission in redditor.submissions.new(limit=limit):
        posts.append({
            "text": submission.title + " " + (submission.selftext or ""),
            "subreddit": str(submission.subreddit),
            "url": submission.url
        })

    for comment in redditor.comments.new(limit=limit):
        comments.append({
            "text": comment.body,
            "subreddit": str(comment.subreddit),
            "url": f"https://reddit.com{comment.permalink}"
        })

    return posts, comments

def build_user_persona(posts, comments):
    """Analyze content and build a persona dictionary with citations."""
    persona = {}
    citations = []

    all_items = posts + comments
    all_subs = [item["subreddit"] for item in all_items]
    top_subs = Counter(all_subs).most_common(5)
    persona["Top Subreddits"] = [sub for sub, _ in top_subs]

    all_text = " ".join(item["text"] for item in all_items)
    words = [word.lower() for word in TextBlob(all_text).words if word.isalpha()]
    keywords = Counter(words).most_common(10)
    persona["Frequent Words"] = [word for word, _ in keywords]

    sentiments = [TextBlob(item["text"]).sentiment.polarity for item in all_items]
    avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else 0
    if avg_sentiment > 0.1:
        persona["Tone"] = "Positive"
    elif avg_sentiment < -0.1:
        persona["Tone"] = "Negative"
    else:
        persona["Tone"] = "Neutral"

    avg_len = sum(len(item["text"]) for item in all_items) / len(all_items)
    persona["Writing Style"] = "Verbose" if avg_len > 200 else "Concise"

    for trait in persona:
        values = persona[trait]
        if isinstance(values, list):
            example = next((item for item in all_items if any(word in item["text"].lower() for word in values)), None)
        else:
            example = all_items[0] if all_items else None
        if example:
            citations.append((trait, example["text"][:150], example["url"]))

    return persona, citations

def save_persona_to_txt(username, persona, citations):
    """Save the structured user persona with citations to a text file."""
    filename = f"{username}_persona.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"USER PERSONA FOR u/{username}\n\n")
        f.write("-----------------------------------------\n")
        f.write(f"Name: u/{username}\n")
        f.write("Occupation: Inferred from Reddit activity\n")
        f.write("Location: Inferred from subreddits\n")
        f.write("Status: Inferred from comment content\n")
        f.write("Archetype: Derived from themes (e.g. Explorer, Analyst)\n")
        f.write("-----------------------------------------\n\n")

        f.write("MOTIVATIONS:\n")
        f.write("- Based on common themes in posts and comments\n")
        cited = next((c for c in citations if c[0] == "Frequent Words"), None)
        if cited:
            f.write(f"📌 Cited: \"{cited[1]}...\" → {cited[2]}\n\n")

        f.write("FRUSTRATIONS:\n")
        f.write("- Common complaints or emotional language\n")
        cited = next((c for c in citations if c[0] == "Tone"), None)
        if cited:
            f.write(f"📌 Cited: \"{cited[1]}...\" → {cited[2]}\n\n")

        f.write("BEHAVIOR & HABITS:\n")
        f.write(f"- Posts in {', '.join(persona['Top Subreddits'])}\n")
        f.write(f"- Writing style: {persona['Writing Style']}\n")
        cited = next((c for c in citations if c[0] == "Top Subreddits"), None)
        if cited:
            f.write(f"📌 Cited: \"{cited[1]}...\" → {cited[2]}\n\n")

        f.write("GOALS & NEEDS:\n")
        f.write("- Derived from language like 'I want', 'I hope', etc.\n")
        cited = next((c for c in citations if c[0] == "Writing Style"), None)
        if cited:
            f.write(f"📌 Cited: \"{cited[1]}...\" → {cited[2]}\n\n")

        f.write("PERSONALITY:\n")
        f.write("- Based on sentiment, writing tone and subreddit types\n")
        f.write(f"- Tone: {persona['Tone']}\n")
        f.write(f"- Style: {persona['Writing Style']}\n")
        cited = next((c for c in citations if c[0] == "Tone"), None)
        if cited:
            f.write(f"📌 Cited: \"{cited[1]}...\" → {cited[2]}\n\n")

    return filename

def main(profile_url):
    """Main function to drive Reddit scraping and persona generation."""
    username = extract_username(profile_url)
    posts, comments = fetch_user_data(username)
    persona, citations = build_user_persona(posts, comments)
    file_path = save_persona_to_txt(username, persona, citations)
    files.download(file_path)
    return file_path

# Example usage:
# profile_url = "https://www.reddit.com/user/Hungry-Move-6603/"
# main(profile_url)


In [33]:
# Example usage:
profile_url = "https://www.reddit.com/user/kojied/"
main(profile_url)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'kojied_persona.txt'